In [1]:
!nvidia-smi

Sat Jan 25 16:17:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.57.02              Driver Version: 566.03         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 Ti     On  |   00000000:01:00.0  On |                  N/A |
|  0%   41C    P8              5W /  165W |     611MiB /  16380MiB |     18%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
import os
import torch
import transformers
import peft
import datasets
import evaluate
import time
assert torch.cuda.is_available(), "you need cuda for this part"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device

device(type='cuda', index=0)

In [3]:
dataset_path = "/app/datasets/oberon/docs/bb_ru"
model_name = "Qwen/Qwen2.5-Coder-3B-Instruct"
#model_name = 'MTSAIR/Cotype-Nano'

EPOCHS = 7
PACKED=False

In [4]:
only_model_name = model_name.split("/")[-1].replace(':', "_")
dataset_name = dataset_path.split("/")[-1]
sft_model_path = f"/app/models/{dataset_name}_{only_model_name}"

In [5]:
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)
peft_config = peft.LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "mlp.down_proj",
        "self_attn.k_proj",
        "self_attn.o_proj",
        "mlp.up_proj",
        "self_attn.v_proj",
        "mlp.gate_proj",
        "self_attn.q_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=peft.TaskType.CAUSAL_LM
)


In [6]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

In [7]:
def load_texts():
    file_names = [] 
    for subdir, dirs, files in os.walk(dataset_path):
        if "/Rsrc/" in subdir:
            continue
        for file in files:
            file_names.append(os.path.join(subdir, file))
    texts = []
    for f in file_names:
        with open(f, 'r', encoding='utf-8') as file:
            texts.append(file.read() + tokenizer.eos_token)
    return file_names,texts
names, texts = load_texts()



In [8]:
#names = [n[len(dataset_path)+1:] for n in names]
#text_dataset = datasets.Dataset.from_dict({"texts":texts,"names" : names})

#text_dataset.push_to_hub('hodza/BlackBox.Shkola.2014')

In [16]:
block_size = 512  # Максимальная длина последовательности
stride = 32      # Шаг перекрытия    
def tokenize_with_stride(texts):
    tokenized_data = tokenizer(
        texts,
        max_length=block_size,
        truncation=True,
        stride=stride,
        return_overflowing_tokens=True,
        padding="max_length",  # Дополняем до block_size
        return_tensors="pt",   # Возвращаем тензоры PyTorch
        add_special_tokens = False,
    )
    tokenized_data["labels"] = tokenized_data["input_ids"]
    return datasets.Dataset.from_dict(tokenized_data).train_test_split(test_size=0.1,shuffle=False)
    
lm_datasets = tokenize_with_stride(texts)
lm_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'overflow_to_sample_mapping', 'labels'],
        num_rows: 5986
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'overflow_to_sample_mapping', 'labels'],
        num_rows: 666
    })
})

In [17]:
print(tokenizer.decode(lm_datasets["train"][3]["input_ids"], skip_special_tokens=False))

Сборка Блэкбокса для школьных уроков информатики

«Блэкбокс  практически идеальная среда для обучения программированию.»
Т.Евсикова, научный сотрудник, 
Институт математики и информатики, Вильнюс, Литва

«... школьная сборка  это блестящая работа, лучше я еще не видел. 
Для меня главное, что показан путь, каким надо двигаться в рамках моих учебных дел. 
Это очень ценно и для развивающихся учителей: они сами будут достраивать <сборку> 
в нужном направлении. Никаких специальных институтов не нужно.»
В.В.Лаптев, профессор, 
Астраханский гос. тех. университет

«... скачал вашу сборку и как в сказку попал ... 
Ввод через любое выделение в документе, дружелюбная гипернавигация, 
<ключевые> слова <можно писать> на русском, мастер нового приложения с подсказками ...»
Александр Шостак, студент,
специальность "экономическая информатика"

Основа школьной сборки  уникальная современная бесплатная и открытая среда программирования Блэкбокс (BlackBox Component Builder), воплотившая полувековой (счит

In [11]:
model = transformers.AutoModelForCausalLM.from_pretrained(model_name, device_map=device,quantization_config=bnb_config,)
model._hf_peft_config_loaded = True  # silence a warning from HF trainer

Loading checkpoint shards: 100%|██████████| 2/2 [00:45<00:00, 22.56s/it]


In [12]:
model = peft.get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 29,933,568 || all params: 3,115,872,256 || trainable%: 0.9607


In [13]:
import logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(message)s')

In [15]:
metric = evaluate.load("bleu")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

2025-01-25 16:20:52,081 Starting new HTTPS connection (1): s3.amazonaws.com:443
2025-01-25 16:20:52,725 https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/metrics/evaluate-metric/bleu/evaluate-metric/bleu.py HTTP/11" 404 0
2025-01-25 16:20:52,731 Starting new HTTPS connection (1): huggingface.co:443
2025-01-25 16:20:53,078 https://huggingface.co:443 "HEAD /spaces/evaluate-metric/bleu/resolve/v0.4.3/bleu.py HTTP/11" 404 0
2025-01-25 16:20:53,082 Starting new HTTPS connection (1): huggingface.co:443
2025-01-25 16:20:53,438 https://huggingface.co:443 "HEAD /spaces/evaluate-metric/bleu/resolve/main/bleu.py HTTP/11" 200 0
2025-01-25 16:20:53,461 Starting new HTTPS connection (1): github.com:443
2025-01-25 16:20:53,856 https://github.com:443 "HEAD /tensorflow/nmt/raw/master/nmt/scripts/bleu.py HTTP/11" 302 0
2025-01-25 16:20:53,857 Starting new HTTPS connection (1): raw.githubusercontent.com:443
2025-01-25 16:20:54,159 https://raw.githubusercontent.com:443 "HEAD /tensorflow

In [18]:
timestr = time.strftime("%Y%m%d-%H%M%S")
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
training_args = transformers.TrainingArguments(
    output_dir=f"./results/{dataset_name}_{only_model_name}/{timestr}/",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    bf16=torch.cuda.get_device_capability(torch.cuda.current_device())[0] >= 8,  
    gradient_accumulation_steps=4,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    #save_total_limit=2,
    logging_dir='./logs',
    logging_steps=32,
    warmup_steps=100, 
    #eval_steps=32, 
    #max_steps=512,
    
    #load_best_model_at_end=True,
)
trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
    data_collator=data_collator,
    #compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()
#trainer.evaluate()

Epoch,Training Loss,Validation Loss


In [125]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 5.12


In [18]:
final_path = sft_model_path + f"_{timestr}"
model.save_pretrained(final_path)
tokenizer.save_pretrained(final_path)

2025-01-23 14:16:24,142 Resetting dropped connection: huggingface.co
2025-01-23 14:16:24,795 https://huggingface.co:443 "HEAD /MTSAIR/Cotype-Nano/resolve/main/config.json HTTP/11" 200 0
2025-01-23 14:16:24,982 https://huggingface.co:443 "HEAD /MTSAIR/Cotype-Nano/resolve/main/config.json HTTP/11" 200 0


('/app/models/bb_ru_Cotype-Nano_20250123-100550/tokenizer_config.json',
 '/app/models/bb_ru_Cotype-Nano_20250123-100550/special_tokens_map.json',
 '/app/models/bb_ru_Cotype-Nano_20250123-100550/vocab.json',
 '/app/models/bb_ru_Cotype-Nano_20250123-100550/merges.txt',
 '/app/models/bb_ru_Cotype-Nano_20250123-100550/added_tokens.json',
 '/app/models/bb_ru_Cotype-Nano_20250123-100550/tokenizer.json')

In [25]:
final_path

'/app/models/bb_ru_Cotype-Nano_20250123-100550'

In [22]:
reference_model = transformers.AutoModelForCausalLM.from_pretrained(model_name, device_map=device,quantization_config=bnb_config,)

2025-01-25 20:38:24,715 Resetting dropped connection: huggingface.co
2025-01-25 20:38:25,242 https://huggingface.co:443 "HEAD /Qwen/Qwen2.5-Coder-3B-Instruct/resolve/main/config.json HTTP/11" 200 0
Loading checkpoint shards: 100%|██████████| 2/2 [00:45<00:00, 22.93s/it]
2025-01-25 20:39:11,632 https://huggingface.co:443 "HEAD /Qwen/Qwen2.5-Coder-3B-Instruct/resolve/main/generation_config.json HTTP/11" 200 0


In [23]:
from IPython.display import HTML, display
table_template = """<table style="border:1px solid black" >
  <tr>
    <th style="text-align: center; border:1px solid black">PROMPT</th>
    <th style="text-align: center; border:1px solid black">BEFORE</th>
    <th style="text-align: center; border:1px solid black">AFTER</th>
  </tr>
{}
</table>"""

row_template = '''  <tr>
    <td style="width:20%; border:1px solid black"><pre align="left">`{}`</pre></td>
    <td style="width:40%; border:1px solid black"><pre align="left">{}</pre></td>
    <td style="width:40%; border:1px solid black"><pre align="left">{}</pre></td>
  </tr>'''

def prompt_to_chat(prompt):
    messages = [
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    return text

def infer(model, prompt, l=100, use_chat = True, temperature=0.4, top_p = 0.8):
    if use_chat:
        prompt = prompt_to_chat(prompt)
    model_inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=l,
        temperature=temperature, 
        top_p=top_p,
        do_sample=True ,  
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=False)[0]
    return response


prompts =  [
    'Как в BlackBox можно вывести данные на консоль?', 
    'Какой тип данных в Component Pascal используется для хранения целых чисел?', 
    'МОДУЛЬ i21егэDemo2010C4ru;', 
    'Component Pascal is Oberon microsystems refinement of?', 
    'Log.String(', 
    'Типом целой константы является'
]  # feel free to add a few more that are not 100% assiciated with Python

In [24]:
def draw_diff(use_chat,l):
    print(f"USING CHAT TEMPLATE = {use_chat}")
        
    baseline = [infer(reference_model, p, l = l, use_chat=use_chat) for p in prompts]
    check = [infer(model, p, l = l, use_chat=use_chat) for p in prompts]
    rows = []
    for i, prompt in enumerate(prompts):
        # replace placeholders in the format() arguments
        rows.append(row_template.format(prompt, baseline[i], check[i]))
    display(HTML(table_template.format('\n'.join(rows))))

In [25]:
draw_diff(True, 256)

USING CHAT TEMPLATE = True


PROMPT,BEFORE,AFTER
`Как в BlackBox можно вывести данные на консоль?`,В BlackBox (также известном как Prometheus) вы можете выводить данные на консоль с помощью различных методов. Вот несколько распространенных способов: 1. **Исп,"В BlackBox, чтобы вывести данные на консоль, достаточно вызвать процедуру StdLog.WriteString или StdLog.Ln. Например: MODULE i21примВывод"
`Какой тип данных в Component Pascal используется для хранения целых чисел?`,В Component Pascal используется тип данных `Integer` для хранения целых чисел.<|im_end|>,В Component Pascal целые числа представлены типами INTEGER и SHORTINT. Тип INTEGER может хранить значения от -2147483648 до 2
`МОДУЛЬ i21егэDemo2010C4ru;`,It looks like you've provided a module name in a specific format: `i21егэDemo2010C4ru`. This appears to be a custom naming convention or identifier,"Модуль переведен на русский язык, но не проверялся на ошибки. В частности, не проверялась работа с файлами. Возможно, нужна"
`Component Pascal is Oberon microsystems refinement of?`,Component Pascal is a programming language developed by Oberon Microsystems. It is a subset of the Oberon programming language and is used for developing software components. Component Pascal is designed to be more efficient and,"Component Pascal is a variant of the Oberon programming language that was developed by the Oberon microsystems company. It was designed to be a more efficient and flexible implementation of the Oberon language,"
`Log.String(`,"It looks like you're trying to use the `Log.String` method, which is part of a logging framework. However, I don't have enough context to provide a specific implementation or explanation.",It looks like you're trying to use a command or function in a way that isn't supported by the Log module. The Log module is part of the BlackBox Component Builder's standard library and
`Типом целой константы является`,"Типом целой константы в большинстве языков программирования является ""целое число"". Это может быть 8-, 16-, 32- или",Типом целой константы является INTEGER или LONGINT в зависимости от размера числа. Примеры: 1234567890 -12


In [26]:
draw_diff(False, 39)

USING CHAT TEMPLATE = False


PROMPT,BEFORE,AFTER
`Как в BlackBox можно вывести данные на консоль?`,В BlackBox можно вывести данные на консоль с помощью различных методов. Вот несколько распространенных способов: 1. **Использование `print` функции:,"Как сделать, чтобы текст был выведен в окно, а не в консоль? В BlackBox есть два модуля для работы с текстом: TextModels и TextViews"
`Какой тип данных в Component Pascal используется для хранения целых чисел?`,В Component Pascal тип данных для хранения целых чисел обычно называется: A) Integer B) Real C) String D) Boolean Правильный ответ:,"(Числа с плавающей точкой тоже могут быть целыми, например, 1.0, 2.5, 3.14159"
`МОДУЛЬ i21егэDemo2010C4ru;`,1. Экспорт модуля: export module i21egegDemo2010C4ru; 2. Импорт модуля: import i,"(* 2010-09-28, Ф.В.Ткачев *) ПОДКЛЮЧИТЬ Вывод := i21"
`Component Pascal is Oberon microsystems refinement of?`,? A) C B) Java C) Pascal D) C++ Answer: A) C Oberon microsystems was a company that developed a programming language called Oberon,A. Smalltalk B. Modula-2 C. Component Pascal D. C++ E. Java F. Visual Basic G. Python H. Ruby I. Perl J. Lisp K.
`Log.String(`,"“error”, err.Error()) return } defer func() { if err := recover(); err != nil { log.Error().Interface(“error”, err).Msgf(",""" "" ); Log.Int( v.len ); Log.Ln END; Log.Ln; Log.String(""End of list""); Log.Ln END Do;"
`Типом целой константы является`,": 1. 0 2. 1 3. -1 4. 1000 Правильный ответ: 1, 2, 3,","целое число, которое может быть представлено в диапазоне от MIN(INTEGER) до MAX(INTEGER). В зависимости от контекста целая константа может быть интер"


In [27]:
print(infer(model, 'Ты помошник по среде Component Pascal. Ответь на вопрос: Как мне вывести строку hello в окно лога?', 256, use_chat = True))

Чтобы вывести строку "hello" в окно лога, выполните следующий код:

	StdLog.String("hello"); StdLog.Ln

Этот код экспортируется модулем StdLog, который должен быть импортирован в начале программы. Команда String используется для вывода текста в лог. Команда Ln используется для перехода на новую строку в логе. Команды могут быть вызваны из любого места программы.

Если вы хотите, чтобы строка была выведена сразу после запуска программы, то можно использовать команду:

	StdLog.Open;  StdLog.String("hello");  StdLog.Ln;

В этом случае программа открывает лог, а затем выводит строку "hello" и переходит на новую строку. Если вы хотите, чтобы программа не открывала лог, то достаточно убрать команду Open.<|im_end|>


In [8]:
checkpoint = transformers.AutoModelForCausalLM.from_pretrained("./results/20250122-220441/checkpoint-1220/", device_map=device)

In [14]:
print(infer(checkpoint, 'Какой тип использовать для хранения целых чисел?', 256, use_chat = True))

Для целых чисел обычно используют тип INTEGER. Однако, если значения могут быть только некоторым предопределенным количеством значений, следует использовать тип SHORTINT, чтобы избежать возможных потерь точности, которые могут возникнуть при делении на 2. В некоторых случаях лучше использовать LONGINT.

Все последующие примеры использует LONGINT в качестве основного типа. Если результаты вычислений становятся непредсказуемыми, если используется другой тип.

Пример:
	
	PROCEDURE Ddouble* (a, b: LONGINT);
	BEGIN
		DIAGNOSTIC;
		StdLog.Nl;
		StdLog.String("Ddouble "); StdLog.Int(a + b, 0); StdLog.Ln;
		StdLog.String(DefaCT(a + b)); StdLog.Ln
	END Ddouble;

В этом примере использовано значение эффекта (defacto) стандартного модуля Math, который включает в себя итоговое значение.
После применения команд DIAGNOSTIC (см. ниже), он будет выровнен по левому краю, а не
